# Data Engineer Course on DataCamp Project

![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [1]:
#importing libraries
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('bank_marketing.csv') #Loading whole file
df

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional.course,no,yes,no,cellular,nov,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,41185,56,retired,married,university.degree,no,yes,no,cellular,nov,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,41186,44,technician,married,professional.course,no,no,no,cellular,nov,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


#### Client Database

In [3]:
#columns name from df which is used for client dataframe
client_cols = ['client_id', 'age', 'job','marital','education','credit_default','housing','loan'] 
client = df[client_cols].copy()
client = client.rename(columns={'client_id':'id'})
client

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional.course,no,yes,no
41184,41184,46,blue-collar,married,professional.course,no,no,no
41185,41185,56,retired,married,university.degree,no,yes,no
41186,41186,44,technician,married,professional.course,no,no,no


In [4]:
#Data Cleaning
client['job'] = client['job'].str.strip('.')
client['education'] = client['education'].str.replace(".", "_").replace('unknown', np.nan)
client['credit_default'] = client['credit_default'].replace("unknown", np.nan)

C:\Users\Dhruv\AppData\Local\Temp/ipykernel_16632/1138955519.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  client['education'] = client['education'].str.replace(".", "_").replace('unknown', np.nan)


In [5]:
# Define the mapping for boolean conversion for specific columns
#found 'yes', 'no' using client['loan'].unique()
column_mapping = {'credit_default': {'yes': True, 'no': False},
                  'housing': {'yes': True, 'no': False},
                  'loan': {'yes': True, 'no': False}}
client = client.replace(column_mapping)

In [6]:
#Assigning boolean datatype
data_type={'credit_default': bool, 'housing': bool, 'loan': bool}
client= client.astype(data_type)
client #This is actual client database

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,False,False,False
1,1,57,services,married,high_school,True,False,False
2,2,37,services,married,high_school,False,True,False
3,3,40,admin,married,basic_6y,False,False,False
4,4,56,services,married,high_school,False,False,True
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional_course,False,True,False
41184,41184,46,blue-collar,married,professional_course,False,False,False
41185,41185,56,retired,married,university_degree,False,True,False
41186,41186,44,technician,married,professional_course,False,False,False


#### Campaign Database

In [7]:
campaign_cols = ['client_id', 'campaign', 'duration','pdays','previous','poutcome','y','day', 'month']
campaign = df[campaign_cols].copy()
campaign = campaign.rename(columns={'campaign':'number_contacts', 'duration': 'contact_duration', 'previous':'previous_campaign_contacts', 'poutcome':'previous_outcome', 'y':'campaign_outcome'})

In [8]:
#Mapping values
column_mapping = {'previous_outcome': {'success': 1, 'failure': 0, 'nonexistent':np.nan},
                  'campaign_outcome': {'yes': 1, 'no': 0}}
campaign = campaign.replace(column_mapping)

In [9]:
data_type={'previous_outcome': bool, 'campaign_outcome': bool}
campaign= campaign.astype(data_type)

In [10]:
#Inserting '1' to campaign_id
campaign.insert(0, 'campaign_id', 1)

In [11]:
campaign['year']= 2022 #Assigning '2022' to campaign['year']
campaign['month'] = pd.to_datetime(campaign['month'], format='%b').dt.month
campaign['last_contact_date'] = pd.to_datetime(campaign[['year', 'month', 'day']]).dt.strftime('%Y-%m-%d')
campaign.drop(['day', 'month', 'year'], axis=1, inplace=True) #Dropping unnecessary columns
campaign #This is actual campaign database

,campaign_id,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,1,0,1,261,999,0,True,False,2022-05-13
1,1,1,1,149,999,0,True,False,2022-05-19
2,1,2,1,226,999,0,True,False,2022-05-23
3,1,3,1,151,999,0,True,False,2022-05-27
4,1,4,1,307,999,0,True,False,2022-05-03
...,...,...,...,...,...,...,...,...,...
41183,1,41183,1,334,999,0,True,True,2022-11-30
41184,1,41184,1,383,999,0,True,False,2022-11-06
41185,1,41185,2,189,999,0,True,False,2022-11-24
41186,1,41186,1,442,999,0,True,True,2022-11-17


#### Economics dataframe

In [12]:
cols = ['client_id', 'emp_var_rate', 'cons_price_idx','euribor3m','nr_employed']
economics = df[cols].copy()
economics = economics.rename(columns={'euribor3m':'euribor_three_months','nr_employed':'number_employed'})
economics #This is actual economics database

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0
...,...,...,...,...,...
41183,41183,-1.1,94.767,1.028,4963.6
41184,41184,-1.1,94.767,1.028,4963.6
41185,41185,-1.1,94.767,1.028,4963.6
41186,41186,-1.1,94.767,1.028,4963.6


#### Creating CSV file

In [13]:
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)

#### Creating Table using SQL operations

In [14]:
# client_table
client_table = """
CREATE TABLE client (
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);

\copy client FROM 'client.csv' DELIMITER ',' CSV HEADER;
"""

In [15]:
# campaign_table
campaign_table = """
CREATE TABLE campaign (
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL REFERENCES client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE
);

\copy campaign FROM 'campaign.csv' DELIMITER ',' CSV HEADER;
"""

In [16]:
# economics_table
economics_table = """
CREATE TABLE economics (
    client_id SERIAL REFERENCES client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);

\copy economics FROM 'economics.csv' DELIMITER ',' CSV HEADER;
"""